In [7]:
import pickle
import pandas as pd
from os import listdir
from os.path import isfile, join, isdir

import torch
import torch.multiprocessing

from helpers.tools import *
from helpers.tools import lagged_batch_corr
from model.model_wrapper import Architecture_PL


In [ ]:
# Complete summary of all parameter searches that were performed. 

# Naming conventions changed towards to end of the paper: 
#SL: base
#ML: big_base_mirror
#SNL: poly
#MNL: big_poly
#LNL: nl
#XLNL: big_nl

In [36]:
def load_res_folder(mypath = "experimental_results/synthetic_data_1_grid_results/std/"):

    onlyfiles = [mypath + f for f in listdir(mypath) if isfile(join(mypath, f))]
    onlyfiles = sorted([x for x in onlyfiles if "summary" in x])
    res = pd.concat([pd.read_csv(x, index_col=0) for x in onlyfiles], axis=1).T
    return  res

# Experiment 1

In [71]:
a = load_res_folder("experimental_results/synthetic_data_1_grid_results/run_1/")
b = load_res_folder("experimental_results/synthetic_data_1_grid_results/run_2/")

In [74]:
res = load_res_folder("experimental_results/synthetic_data_1_grid_results/std/")
res.loc[res["model"].isnull(), "model"] = "small.yaml"
# Provide some summary statistics over the runs.
res = res[["model.model_type", "data.ds_name",'Test1_Auroc', 'Test2_AUROC']]
res["Test1_Auroc"] = res["Test1_Auroc"].astype(float)
res["Test2_AUROC"] = res["Test2_AUROC"].astype(float)

res.groupby(["data.ds_name", "model.model_type"]).mean()

HP                                Test1_Auroc  Test2_AUROC
data.ds_name    model.model_type                          
base            bidirectional        0.999499     0.999961
                convM                0.999336     0.999894
                mlp                  0.999328     0.999933
                transformer          0.999534     0.999968
                unidirectional       0.999558     0.999948
big_base_mirror bidirectional        0.613235     0.626371
                convM                0.506232     0.513293
                mlp                  0.499207     0.498309
                transformer          0.600757     0.621380
                unidirectional       0.599960     0.621146
big_nl          bidirectional        0.709532     0.748808
                convM                0.503338     0.506151
                mlp                  0.501747     0.498722
                transformer          0.518067     0.517684
                unidirectional       0.659285     0.688653
big_poly        bidirectional        0.547126     0.555322
                convM                0.518390     0.518896
                mlp                  0.520201     0.524884
                transformer          0.537860     0.545076
                unidirectional       0.784930     0.828722
nl              bidirectional        0.916400     0.936746
                convM                0.912467     0.934696
                mlp                  0.905332     0.932553
                transformer          0.915942     0.934492
                unidirectional       0.915071     0.936571
poly            bidirectional        0.932779     0.948389
                convM                0.931417     0.948634
                mlp                  0.930141     0.950117
                transformer          0.934952     0.955290
                unidirectional       0.934503     0.954934

# Experiment 1 - Joint

In [75]:
c = load_res_folder("experimental_results/synthetic_data_1_joint_grid_results/splitted_run_1/")

In [76]:
res = load_res_folder("experimental_results/synthetic_data_1_joint_grid_results/std/")

# Provide some summary statistics over the runs.
res = res[["model.model_type", "model",'Test1_Auroc', 'Test2_AUROC']]
res["Test1_Auroc"] = res["Test1_Auroc"].astype(float)
res["Test2_AUROC"] = res["Test2_AUROC"].astype(float)

res.groupby(["model", "model.model_type"]).mean()

HP                            Test1_Auroc  Test2_AUROC
model       model.model_type                          
big.yaml    bidirectional        0.952198     0.962480
            convM                0.867127     0.885812
            mlp                  0.751937     0.760817
            transformer          0.969769     0.977339
            unidirectional       0.946323     0.959416
medium.yaml bidirectional        0.906445     0.920411
            convM                0.869493     0.887307
            mlp                  0.915602     0.928056
            transformer          0.938883     0.949564
            unidirectional       0.894770     0.906847

# Experiment 2

In [62]:
res.groupby(["model", "model.model_type"]).mean()

HP                            Test1_Auroc  Test2_AUROC
model       model.model_type                          
big.yaml    bidirectional        0.952198     0.962480
            convM                0.867127     0.885812
            mlp                  0.751937     0.760817
            transformer          0.969769     0.977339
            unidirectional       0.946323     0.959416
medium.yaml bidirectional        0.906445     0.920411
            convM                0.869493     0.887307
            mlp                  0.915602     0.928056
            transformer          0.938883     0.949564
            unidirectional       0.894770     0.906847

# Experiment Kuramoto

In [78]:
e = load_res_folder("experimental_results/kuramoto_grid_results/splitted_run_1/")

In [82]:
res = load_res_folder("experimental_results/kuramoto_grid_results/std/")

# Provide some summary statistics over the runs.
res = res[["model.model_type",'Test1_Auroc']]
res["Test1_Auroc"] = res["Test1_Auroc"].astype(float)

res.groupby([ "model.model_type"]).mean()

HP,Test1_Auroc
model.model_type,
bidirectional,0.852495
convM,0.954027
mlp,0.824648
transformer,0.888369
unidirectional,0.905020


# Zero shot

In [14]:
# We use the best performing checkpoints from all std runs here. They can be downloaded here: 

# Load the best models: 
best = pd.read_csv("experimental_results/synthetic_data_1_joint_grid_results/std/summary_big.csv", index_col=0).T

stack = []
for x in best["model.model_type"].unique():
    stack.append(best.loc[best["model.model_type"] == x].sort_values("Test1_Auroc", ascending=False)[:1])
pd.concat(stack)

HP,seed,hparams_search,model.optimizer_lr,data.batch_size,model.weight_decay,model.model_type,model.corr_regularization,corr_input,regression_head,data.ds_name,max_lags,n_vars,model,Test1_Auroc,Test1_MSE,Test2_AUROC,Test2_MSE
6,43,std_runs_slurm.yaml,0.001,16,0.01,convM,True,True,False,joint,3,5,big.yaml,0.871109,0.03637622,0.89227587,0.03218321
9.1,50,std_runs_slurm.yaml,0.001,64,0,transformer,False,True,False,joint,3,5,big.yaml,0.971022,0.012118887,0.97848606,0.009607594
7.2,49,std_runs_slurm.yaml,0.001,16,0.01,bidirectional,False,True,False,joint,3,5,big.yaml,0.96785104,0.013725153,0.97649235,0.009861699
4.3,42,std_runs_slurm.yaml,0.0001,64,0,mlp,True,True,False,joint,3,5,big.yaml,0.791368,0.046149682,0.7982332,0.043174013
8.4,50,std_runs_slurm.yaml,0.001,64,0.01,unidirectional,False,True,False,joint,3,5,big.yaml,0.96973234,0.013177577,0.9781359,0.009636639


In [15]:
# Saved best performing checkpoints that resulted in the scores above.

mlp = "pretrained_weights/mlp.ckpt"
uni =  "pretrained_weights/unidirectional.ckpt"
bi=  "pretrained_weights/bidirectional.ckpt"
conv =  "pretrained_weights/convMixer.ckpt"
trf =  "pretrained_weights/transformer.ckpt"
best = [mlp, uni, bi, conv, trf]

auroc = torchmetrics.classification.BinaryAUROC()

In [12]:
res = []
data = load_river_data()
# create proper padding
a= torch.concat([data[0][0,:,:], torch.normal(0, 0.1, (len(data[0][0]),2))], axis=1)
a = a.unsqueeze(0)
corr = lagged_batch_corr(a,3)


for x in best:
    model = Architecture_PL.load_from_checkpoint(x)
    M = model.model
    M = M.to("cpu")
    M = M.eval()
    pred = torch.sigmoid(M((a[:,:600,:], corr)))[0,:3,:3, -1]
    res.append(auroc(pred, data[1]))

In [13]:
pd.DataFrame(res, index = ["MLP", "uGRU", "bGRU", "CM", "Trf"])

,0
MLP,tensor(0.4250)
uGRU,tensor(1.0000)
bGRU,tensor(0.5000)
CM,tensor(0.6000)
Trf,tensor(1.0000)
